In [1]:
# CNN  
zipFilePath = '/content/drive/MyDrive/산대특 공유폴더/horse-or-human.zip'

In [2]:
import urllib.request
import zipfile

In [4]:
target_dir = '/content/drive/MyDrive/Colab Notebooks'

In [6]:
# 인터넷에서 target으로파일 read
# urllib.request.urlretrieve(zipFilePath,target_dir)

In [7]:
zip_data = zipfile.ZipFile(zipFilePath,'r')
zip_data.extractall(target_dir)
zip_data.close()

In [8]:
horses_path = '/content/drive/MyDrive/Colab Notebooks/horses'
humans_path = '/content/drive/MyDrive/Colab Notebooks/humans'

In [9]:
import os
import glob

In [14]:
horse_lists = glob.glob(horses_path+"/*.png")
human_lists = glob.glob(humans_path+"/*.png")

In [15]:
from tensorflow.keras.preprocessing import image

In [18]:
temp = image.load_img(horse_lists[0],target_size=(255,255))

In [ ]:
image.img_to_array(image.load_img(horse_lists[0],target_size=(255,255)))

In [25]:
# 데이터 확보
horse_datas = [image.img_to_array(image.load_img(horsefile,target_size=(255,255))) for horsefile in horse_lists]

In [26]:
human_datas = [image.img_to_array(image.load_img(humanfile,target_size=(255,255))) for humanfile in human_lists]

In [28]:
import numpy as np
np.array(horse_datas).shape,  np.array(human_datas).shape

((500, 255, 255, 3), (527, 255, 255, 3))

In [31]:
y = [0]*500 + [1]*527

In [32]:
x = horse_datas + human_datas

In [33]:
len(x), len(y)

(1027, 1027)

In [35]:
# 표준화
x = np.array(x).astype('float32') / 255.0

In [36]:
# 잘섞고 잘 분리
from sklearn.model_selection import train_test_split

In [37]:
x_train,x_test,y_train,y_test =  train_test_split(x,y,random_state=42)

In [40]:
# 신경망 만들기
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D,Flatten,Dropout,MaxPooling2D

In [42]:
model = tf.keras.models.Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(255,255,3)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.25),

    Conv2D(64,(3,3),activation='relu'),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.25),

    Flatten(),

    Dense(128,activation='relu'),
    Dropout(0.5),
    Dense(1,activation='sigmoid')
])

In [43]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 253, 253, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 251, 251, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 125, 125, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 125, 125, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 123, 123, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 121, 121, 64)      36928     
                                                        

In [44]:
model.compile(loss = tf.keras.losses.binary_crossentropy,
              optimizer='adam', metrics=['acc','mae']
              )

In [ ]:
earlyStop = tf.keras.callbacks.EarlyStopping(patience=5)
hist = model.fit(x_train,y_train,batch_size=32,epochs=100,validation_data=(x_test,y_test),callbacks=[earlyStop])